In [ ]:
!pip install ultralytics
!pip install opencv-python-headless
!pip install albumentations
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu124

In [2]:
import os
import cv2
import shutil
import random
from ultralytics import YOLO

In [5]:
classes = [chr(i) for i in range(ord('A'), ord('Z') + 1)] + [str(i) for i in range(10)]
class_map = {c: idx for idx, c in enumerate(classes)}

In [ ]:
#Convert kaggle dataset to YOLO format
kaggle_base_dir = './raw_data/kaggle_dataset'
kaggle_base_dir2 = './raw_data/kaggle_dataset2'
output_dir = './raw_data/yolo_format'
output_dir2 = './raw_data/yolo_format2'
def convert_to_yolo_format(kaggle_base_dir, output_dir):
    for class_label, class_index in class_map.items():
        folder_path = os.path.join(kaggle_base_dir, class_label)
        for image_file in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_file)
            image = cv2.imread(image_path)

            x_center, y_center, box_width, box_height = 0.5, 0.5, 1.0, 1.0

            label_path = os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}.txt")
            with open(label_path, "w") as f:
                f.write(f"{class_index} {x_center} {y_center} {box_width} {box_height}")
            
            cv2.imwrite(os.path.join(output_dir, image_file), image)

convert_to_yolo_format(kaggle_base_dir, output_dir)
convert_to_yolo_format(kaggle_base_dir2, output_dir2)


In [ ]:
combined_dataset_dir = './datasets'
os.makedirs(combined_dataset_dir, exist_ok=True)
for subset in ['train', 'test', 'valid']:
    os.makedirs(os.path.join(combined_dataset_dir, subset, 'images'), exist_ok=True)
    os.makedirs(os.path.join(combined_dataset_dir, subset, 'labels'), exist_ok=True)

roboflow_base_dir = './raw_data/robo_dataset'
roboflow_train_dir = os.path.join(roboflow_base_dir, 'train')
roboflow_test_dir = os.path.join(roboflow_base_dir, 'test')
roboflow_valid_dir = os.path.join(roboflow_base_dir, 'valid')

def copy_dataset(src_dir, dest_dir):
    for file_type in ['images', 'labels']:
        src_folder = os.path.join(src_dir, file_type)
        dest_folder = os.path.join(dest_dir, file_type)
        for file_name in os.listdir(src_folder):
            src_file = os.path.join(src_folder, file_name)
            dest_file = os.path.join(dest_folder, file_name)
            shutil.copy(src_file, dest_file)

copy_dataset(roboflow_train_dir, os.path.join(combined_dataset_dir, 'train'))
copy_dataset(roboflow_test_dir, os.path.join(combined_dataset_dir, 'test'))
copy_dataset(roboflow_valid_dir, os.path.join(combined_dataset_dir, 'valid'))

kaggle_yolo_dir2 = './raw_data/yolo_format' # Change this to kaggle formated dataset
kaggle_yolo_dir = './raw_data/yolo_format2' # Change this to kaggle formated dataset

def copy_kaggle_dataset(dest_dir):
    all_kaggle_images = [f for f in os.listdir(dest_dir) if f.endswith('.jpg')]
    random.shuffle(all_kaggle_images)
    train_split = int(0.8 * len(all_kaggle_images))
    valid_split = int(0.9 * len(all_kaggle_images))

    kaggle_splits = {
        'train': all_kaggle_images[:train_split],
        'valid': all_kaggle_images[train_split:valid_split],
        'test': all_kaggle_images[valid_split:]
    }

    for subset, images in kaggle_splits.items():
        for image_name in images:
            src_image = os.path.join(dest_dir, image_name)
            dest_image = os.path.join(combined_dataset_dir, subset, 'images', image_name)
            shutil.copy(src_image, dest_image)
            
            label_name = os.path.splitext(image_name)[0] + '.txt'
            src_label = os.path.join(dest_dir, label_name)
            dest_label = os.path.join(combined_dataset_dir, subset, 'labels', label_name)
            if os.path.exists(src_label):
                shutil.copy(src_label, dest_label)

copy_kaggle_dataset(kaggle_yolo_dir)
copy_kaggle_dataset(kaggle_yolo_dir2)


In [ ]:
import cv2
import os
import albumentations as A

# Define augmentations
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=45, p=0.5), 
    A.RGBShift(r_shift_limit=40, g_shift_limit=40, b_shift_limit=40, p=.2),
    A.GaussNoise(p=0.2),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.35,p=.5),
    A.OneOf([
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.2),
        A.Blur(blur_limit=3, p=0.2)
    ]),
], bbox_params=A.BboxParams(format='yolo', min_visibility=0.4))

# Function to load, augment, and save images
def augment_and_save_image(image_path, label_path, output_dir, augmentations_per_image=5):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return

    with open(label_path, 'r') as file:
        bboxes = []
        for line in file:
            parts = line.strip().split()
            class_id = int(parts[0])
            bbox = list(map(float, parts[1:]))
            bboxes.append([*bbox, class_id])

    os.makedirs(output_dir, exist_ok=True)
    img_output_dir = os.path.join(output_dir, 'images')
    lbl_output_dir = os.path.join(output_dir, 'labels')
    os.makedirs(img_output_dir, exist_ok=True)
    os.makedirs(lbl_output_dir, exist_ok=True)

    # Generate augmented images and labels
    for i in range(augmentations_per_image):
        augmented = augmentation_pipeline(image=image, bboxes=bboxes)

        aug_image = augmented['image']
        aug_image_filename = f"{os.path.spliteaut_dir, aug_image_filename)
        cv2.imwrite(aug_image_path, aug_image)

        aug_label_filename = f"{os.path.splitext(os.path.basename(label_path))[0]}_aug_{i}.txt"
        aug_label_path = os.path.join(lbl_output_dir, aug_label_filename)
        with open(aug_label_path, 'w') as file:
            for bbox in augmented['bboxes']:
                class_id = int(bbox[4])
                bbox_str = ' '.join(map(str, bbox[:4]))
                file.write(f"{class_id} {bbox_str}\n")

dataset_dir = './datasets' # Change this to your dataset directory
output_dir = './augmented' # Change this to your output directory

for subset in ['train', 'test', 'valid']:
    image_dir = os.path.join(dataset_dir, subset, 'images')
    label_dir = os.path.join(dataset_dir, subset, 'labels')

    for image_file in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_file)
        label_path = os.path.join(label_dir, os.path.splitext(image_file)[0] + '.txt')
        if os.path.exists(label_path):
            augment_and_save_image(image_path, label_path, os.path.join(output_dir, subset))


In [ ]:
model = YOLO("yolo11n.pt")

results = model.train(data="./data.yaml", epochs=300, imgsz=640, save_period=20)


In [ ]:
#If needed to pause and resume
model = YOLO("./runs/detect/train3/weights/last.pt") # Last trained model
results = model.train(data="./data.yaml", epochs=300, imgsz=640, save_period=20, resume=True)
